In [ ]:
import glob 
import os
from PIL import Image
from PIL import ImageDraw
from PIL import ImageOps
from PIL import ImageFont


In [ ]:
def create_vol_figure_contrast(group,ses,task,contrast):
    
    if task == 'mid':
        display_view = 'coronal'
    
    else:
        display_view = 'axial'

    axial_HC_baseline_path = glob.glob(f'../../../derivatives/task_analysis_volume/visualization/fsleyes_indiv_figures/task-{task}/group-HC_ses-baseline_task-{task}_contrast-{contrast}_threshold-*_n-*_display-{display_view}.png')[0]
    axial_HC_baseline=Image.open(axial_HC_baseline_path,mode='r')

    w,h=axial_HC_baseline.size
    area_no_cmap = (0, 20, w-83, h-25)
    
    if display_view == 'coronal':
        area_no_cmap = (0, 70, w-83, h-90)

    HCb = axial_HC_baseline.crop(area_no_cmap) #crop just the part of the image that you want


    axial_MM_baseline_path = glob.glob(f'../../../derivatives/task_analysis_volume/visualization/fsleyes_indiv_figures/task-{task}/group-MM_ses-baseline_task-{task}_contrast-{contrast}_threshold-*_n-*_display-{display_view}.png')[0]
    axial_MM_baseline=Image.open(axial_MM_baseline_path,mode='r')
    MMb = axial_MM_baseline.crop(area_no_cmap) #crop just the part of the image that you want


    axial_MM_1year_path = glob.glob(f'../../../derivatives/task_analysis_volume/visualization/fsleyes_indiv_figures/task-{task}/group-MM_ses-1year_task-{task}_contrast-{contrast}_threshold-*_n-*_display-{display_view}.png')[0]
    axial_MM_1year=Image.open(axial_MM_1year_path,mode='r')

    area_cmap = (0, 20, w, h-25)
    
    if display_view == 'coronal':
        area_cmap = (0, 70, w, h-90)

    MM1 = axial_MM_1year.crop(area_cmap) #crop just the part of the image that you want


    w_no_cmap,h_no_cmap = HCb.size
    w_cmap,h_cmap = MM1.size

    new_im = Image.new('RGB', ( (w_no_cmap*2+w_cmap) , h_no_cmap) ,(255, 255, 255, 1)) #make a new image
    #paste in the cut out images
    new_im.paste(HCb,(0,0)) 
    new_im.paste(MMb,(w_no_cmap,0))
    new_im.paste(MM1,(w_no_cmap*2,0))
    
    #create paths to output dir if not exist
    derivatives_path = '../../../derivatives'
    nilearn_output_path = os.path.join(derivatives_path, 'task_analysis_volume','visualization','per_contrast_figures',f'task-{task}')
    if not os.path.isdir(nilearn_output_path):
        os.makedirs (nilearn_output_path)

    HCb_threshold = axial_HC_baseline_path.split('threshold-')[1].split('_')[0]
    HCb_part_count = axial_HC_baseline_path.split('_n-')[1].split('_')[0]

    MMb_threshold = axial_MM_baseline_path.split('threshold-')[1].split('_')[0]
    MMb_part_count = axial_MM_baseline_path.split('_n-')[1].split('_')[0]

    MM1_threshold = axial_MM_1year_path.split('threshold-')[1].split('_')[0]
    MM1_part_count = axial_MM_1year_path.split('_n-')[1].split('_')[0]

    new_im.save(f'../../../derivatives/task_analysis_volume/visualization/per_contrast_figures/task-{task}/task-{task}_contrast-{contrast}_HCb_threshold-{HCb_threshold}_HCb_n-{HCb_part_count}_MMb_threshold-{MMb_threshold}_MMb_n-{MMb_part_count}_MM1_threshold-{MM1_threshold}_MM1_n-{MM1_part_count}_nifti_figure.png')

    return [w_no_cmap,display_view]


In [ ]:
def get_labels_dict(task):
    
    if task == 'mid':
        labels_dict={ 'HiRewCue-NeuCue':"High Reward\nCue vs.\nNeutral Cue", #high reward anticipation
                      'HiRewCue-LoRewCue':"High Reward\nCue vs.\nLow Reward Cue", #high vs. low reward anticipation
                      'RewCue-NeuCue':"Reward Cue vs.\nNeutral Cue", #combined reward anticipation
                      'LoRewCue-NeuCue':"Low Reward\nCue vs.\nNeutral Cue", #low reward anticipation -- ABCD
                     
                      'HiLossCue-NeuCue':"High Loss\nCue vs.\nNeutral Cue", #high loss anticipation
                      'HiLossCue-LoLossCue':"High Loss\nCue vs.\nLow Loss Cue", #high vs. low loss anticipation
                      'LossCue-NeuCue':"Loss Cue vs.\nNeutral Cue", #combined loss anticipation
                      'LoLossCue-NeuCue':"Low Loss\nCue vs.\nNeutral Cue", #low loss anticipation -- ABCD
                     
                      'HiRewCue-HiLossCue':"High Reward Cue vs.\nHigh Loss Cue", #high reward vs. high loss anticipation
                      'RewCue-LossCue':"Reward Cue vs.\nLoss Cue", #combined reward vs. loss anticipation

                      'HiRewCue-Baseline':"High Reward\nCue vs.\nBaseline", #high reward anticipation vs. baseline -- paper A4

                      'HiWin-NeuHit':"High Reward vs.\nNeutral Hit", #high reward outcome cp. to neutral hit
                      'Win-NeuHit':"Reward vs.\nNeutral Hit", #combined reward outcome cp. to neutral hit

                      'HiWin-HiNoWin':"High Reward vs.\nHigh Missed Reward", #high reward outcome cp. to high reward miss
                      'Win-NoWin':"Reward vs.\nMissed Reward", #combined reward outcome cp. to combined reward miss

                      'HiLoss-NeuMiss':"High Loss vs.\nNeutral Miss", #high loss cp. to neutral miss
                      'Loss-NeuMiss':"Loss vs.\nNeutral Miss", #combined loss cp. to neutral miss

                      'HiLoss-AvoidHiLoss':"High Loss vs.\nAvoided High Loss", #high loss cp. to high avoid loss
                      'Loss-AvoidLoss':"Loss vs.\nAvoided Loss", #combined loss cp. to combined avoid loss

                      'HiLoss-NeuHit':"High Loss vs.\nNeutral Hit", #high loss cp. to neutral hit
                      'Loss-NeuHit':"Loss vs.\nNeutral Hit", #combined loss cp. to neutral hit 

                      'HiWin-HiLoss':"High Reward vs.\nHigh Loss", #high reward outcome cp. to high loss
                      'Win-Loss':"Reward vs.\nLoss", #combined reward outcome cp. to combined loss 
        }
    
    elif task == 'sst':
        labels_dict={'SuccStop-Go':"Successful\nSTOP vs.\nGO",'UnsuccStop-Go':"Unsuccessful\nSTOP vs.\nGO",'UnsuccStop-SuccStop':"Unsuccessful\nSTOP vs.\nSuccessful\nSTOP"}

    elif task == 'nback':
        labels_dict={'twoback-zeroback': "Two-back vs.\nZero-back"}
    
    return labels_dict

In [ ]:
def create_vol_figure_all(task,w_no_cmap,display_view,contrasts,supplemental):
    
    labels_dict = get_labels_dict(task)
    
    font = ImageFont.truetype("../templates/fonts/G_ari_bd.TTF",40)
    font_heading = ImageFont.truetype("../templates/fonts/G_ari_bd.TTF",50)

    #create paths to output dir if not exist
    derivatives_path = '../../../derivatives'
    nilearn_output_path = os.path.join(derivatives_path, 'task_analysis_volume','visualization','complete_figures',f'task-{task}')
    if not os.path.isdir(nilearn_output_path):
        os.makedirs (nilearn_output_path)

         
    per_contrast_figures_paths = glob.glob(f'../../../derivatives/task_analysis_volume/visualization/per_contrast_figures/task-{task}/task-{task}_contrast-*_HCb_threshold-*_HCb_n-*_MMb_threshold-*_MMb_n-*_MM1_threshold-*_MM1_n-*_nifti_figure.png')
    contrast_paths_dict = {img_path.split('contrast-')[1].split('_')[0]:img_path for img_path in per_contrast_figures_paths}
    contrast_count = len(contrasts)
    
    first_img=Image.open(per_contrast_figures_paths[0],mode='r')
    w,h=first_img.size
    
    margin_height = 35 #height of top margin
    new_im = Image.new('RGB', (w, h*contrast_count+margin_height) ,(0, 0, 0, 1)) #make a new image
   
    #paste in the images
    for i in range(contrast_count):
        contrast = contrasts[i]
        img_path = contrast_paths_dict[contrast]
        img = Image.open(img_path,mode='r')
        new_im.paste(img,(0,margin_height+h*i))
        
        
    #resize to have consistent width of letterhead page
    letterhead_width = 8.5 * 300  #assuming 300 pixels per inch
    left_margin = 350 #width of white margin
    
    #calculate the new height to maintain the aspect ratio
    aspect_ratio = new_im.width/new_im.height
    new_height = int((letterhead_width-left_margin)/aspect_ratio)

    #resize the image
    resized_image = new_im.resize((int((letterhead_width-left_margin)), new_height))
    
    #create left margin image to be added to left of figure
    left_margin_im = Image.new('RGB', (left_margin, new_height) ,(255, 255, 255, 1)) #make a new image

    #paste left margin
    final_im = Image.new('RGB', (int(letterhead_width), new_height) ,(255, 255, 255, 1)) #make a new image
    final_im.paste(left_margin_im,(0,0))
    final_im.paste(resized_image,(left_margin,0)) 

    #get ratios for scaling
    resize_width_ratio = resized_image.width/new_im.width
    resize_height_ratio = resized_image.height/new_im.height
    
    draw = ImageDraw.Draw(final_im)

    top_offset=25
    draw.text(((w_no_cmap*0.5)*resize_width_ratio+left_margin,top_offset),"HC baseline",fill=(255,255,255),font=font_heading,anchor="ma")
    draw.text(((1.5*w_no_cmap)*resize_width_ratio+left_margin,top_offset),"MCC baseline",fill=(255,255,255),font=font_heading,anchor="ma")
    draw.text(((2.5*w_no_cmap)*resize_width_ratio+left_margin,top_offset),"MCC one-year",fill=(255,255,255),font=font_heading,anchor="ma")

    for i in range(contrast_count):
        contrast = contrasts[i]
        label = labels_dict[contrast]
        draw.multiline_text((15,(h*(i+0.5))*resize_height_ratio),f'{label}',fill=(0,0,0),font=font,anchor="la")

    #display complete figure and save it  
    display(final_im)
    
    if supplemental:
        supplemental_figure_path = f'../../../derivatives/task_analysis_volume/visualization/complete_figures/task-{task}/task-{task}_display-{display_view}_figure_supplemental.png'
        final_im.save(supplemental_figure_path)
    else:
        figure_path = f'../../../derivatives/task_analysis_volume/visualization/complete_figures/task-{task}/task-{task}_display-{display_view}_figure.png'
        final_im.save(figure_path)

    return


In [ ]:
def create_figure(task,groups,sessions,supplemental=False):
    
    if task == 'mid':
        #main text figure
        contrasts = [ 'HiRewCue-Baseline', #high reward anticipation vs. baseline -- paper A4
                      'Win-NoWin', #combined reward outcome cp. to combined reward miss -- ABCD
                      'HiLoss-NeuHit', #high loss cp. to neutral hit -- paper O7
                    ]
        if supplemental:
            #supplemental figure
            contrasts = [ 'HiRewCue-NeuCue', #high reward anticipation -- paper A2, ABCD
                          'LoRewCue-NeuCue', #low reward anticipation -- ABCD
                          'RewCue-NeuCue', #combined reward anticipation -- paper A1
                          'HiRewCue-LoRewCue', #high vs. low reward anticipation -- paper A3, ABCD
                          'HiLossCue-NeuCue', #high loss anticipation -- paper A5, ABCD
                          'LoLossCue-NeuCue', #low loss anticipation -- ABCD
                          'HiLossCue-LoLossCue', #high vs. low loss anticipation -- ABCD
                          'HiWin-NeuHit', #high reward outcome cp. to neutral hit -- paper O6
                          'Loss-AvoidLoss', #combined loss cp. to combined avoid loss -- ABCD
                         ]

    elif task == 'sst':
        #main text figure
        contrasts=['SuccStop-Go','UnsuccStop-Go']
        
        if supplemental:
            #supplemental text figure
            contrasts=['UnsuccStop-SuccStop']

    elif task == 'nback':
        #main text figure
        contrasts=['twoback-zeroback']
        
        if supplemental:
            print('supplemental figure not created for nback')
            return
    
    for group in groups:
        for ses in sessions:
            for contrast in contrasts:
                w_no_cmap, display_view = create_vol_figure_contrast(group,ses,task,contrast)

    create_vol_figure_all(task,w_no_cmap,display_view,contrasts,supplemental)
    

In [ ]:
groups=['HC','MM']
sessions=['baseline','1year']
task='mid'
supplemental=False

create_figure(task,groups,sessions,supplemental)